
# Maximal Representative Subsampling


<div class="alert alert- block alert-warning"> <b>Todo:</b> 

- Use unprundes trees

- Try linear svm with C param tuning

- next meeting: 5th september 11h.

</div>

In [38]:
from pathlib import Path
import os
%config IPCompleter.greedy=True

import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
np.seterr(divide = 'ignore')
import random

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn import metrics
from scipy import stats

path = Path(os.getcwd()).parent

## MRS ALGORITHM

In [ ]:
def temp_sample(softmax, temperature, drop):

    EPSILON = 10e-16 # to avoid taking the log of zero
    softmax = (np.array(softmax) + EPSILON).astype('float64')
    
    preds = np.log(softmax) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(drop, preds, 1)

    return probas[0]

In [ ]:
def _bootstrap(df, n):
    train = df.sample(n, replace=True)
    tmp = df.index.isin(train.index)
    test = df[~tmp]
    return train, test

In [31]:
def svc_param_selection(X, y, nfolds):
    
    svc = LinearSVC(max_iter=1000)
    C = [0.001, 0.01, 0.1, 1, 10]
    param_grid = {'C': C}
    grid = GridSearchCV(estimator=svc, iid=True, param_grid=dict(C=C), cv=nfolds)
    grid.fit(X, y)
    grid.best_params_
    return grid.best_params_

In [42]:
def MRS(nonrep, rep, columns, temperature, n_drop, limit, ensemble_size, C=1.0):
    
    auc = []
    
    nonrep['label'] = 1
    rep['label'] = 0
    
    while (nonrep.shape[0] > limit):
    
        nonrep['__preds'] = 0
        rep['__preds'] = 0
    
        nonrep['__count'] = 0
        rep['__count'] = 0
    
        n = min(nonrep.shape[0], rep.shape[0])
        print(n, end=', ')
        
        for _ in range(ensemble_size):

            train_nonrep, test_nonrep = _bootstrap(nonrep, n)
            train_rep, test_rep = _bootstrap(rep, n)
            
            train = pd.concat([train_nonrep, train_rep], sort=False)
            test = pd.concat([test_nonrep, test_rep], sort=False)
            
            svm = LinearSVC(C=C, max_iter=1000)
            clf = CalibratedClassifierCV(svm, cv=5)
            clf.fit(train[columns], train.label)
            
            test['__preds'] = [i+j[1] for i,j in zip(test['__preds'], clf.predict_proba(test[columns]))]
            test['__count'] = [i+1 for i in test['__count']]
            
            df = pd.concat([train.drop_duplicates(subset=train.columns, keep='first', inplace=False), test])
            
            nonrep = df[df.label == 1].copy(deep=True)
            rep = df[df.label == 0].copy(deep=True)
            
            del train_nonrep, test_nonrep, train_rep, test_rep, train, test
        
        oosample = df[df.__count!=0] #predicted at least once
        insample = df[df.__count==0] #only used to train
        oosample['__preds'] = [i/j for i,j in zip(oosample['__preds'], oosample['__count'])]

        auc.append((-n, metrics.roc_auc_score(oosample.label, oosample.__preds)))

        del oosample, insample
        
        drop = df[(df.__count!=0) & (df.label==1)]
        keep = df[(df.__count==0) | (df.label==0)]
        
        drop['removed'] = temp_sample(drop.__preds, temperature, n_drop) 
        
        drop = drop[drop.removed == 0]
        drop.drop('removed', axis=1, inplace=True)
        
        df = pd.concat([drop, keep], sort=True)
        df.reset_index(inplace=True, drop=True)
        del drop, keep
        
        nonrep = df[df.label==1].copy(deep=True)
        rep = df[df.label==0].copy(deep=True)
        
    return auc

### US National Census (Income) <a name="us"></a>

*About this Dataset*

**US Adult Census** (1994) relates income to social factors: 

- *age*: continuous.
- *workclass*: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- *fnlwgt*: continuous.
- *education*: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- *education-num*: continuous.
- *marital-status*: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- *occupation*: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- *relationship*: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- *race*: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- *sex*: Female, Male.
- *capital-gain*: continuous.
- *capital-loss*: continuous.
- *hours-per-week*: continuous.
- *native-country*: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

Each row is labelled as either having a salary greater than ">50K" or "<=50K".

Note: This Dataset was obtained from the UCI repository, it can be found on

https://archive.ics.uci.edu/ml/datasets/census+income, http://mlr.cs.umass.edu/ml/machine-learning-databases/adult/

In [ ]:
columns = ['Age','Workclass','fnlgwt','Education','Education Num','Marital Status',
           'Occupation','Relationship','Race','Sex','Capital Gain','Capital Loss',
           'Hours/Week','Country','Above/Below 50K']

train = pd.read_csv(os.path.join(path, 'data/census_income/adult.data'), names=columns)
test = pd.read_csv(os.path.join(path, 'data/census_income/adult.test'), names=columns)
test = test.iloc[1:]

df = pd.concat([train, test]).copy(deep=True)

del train, test

df.replace(' >50K.', ' >50K', inplace=True)
df.replace(' <=50K.', ' <=50K', inplace=True)

df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

ctg = ['Workclass', 'Sex', 'Education', 'Marital Status', 
       'Occupation', 'Relationship', 'Race', 'Country']
for c in ctg:
    df = pd.concat([df, pd.get_dummies(df[c], 
                                       prefix=c,
                                       dummy_na=False)], axis=1).drop([c],axis=1)

'''Rep: <=50K    37155 ;; >50K     11687'''

df_high = df[df['Above/Below 50K'] == " >50K"].copy(deep=True)
df_low = df[df['Above/Below 50K'] == " <=50K"].copy(deep=True)

df_low = df_low.reindex(np.random.permutation(df_low.index))
df_high = df_high.reindex(np.random.permutation(df_high.index))

rep = pd.concat([df_low.head(5000).copy(deep=True),
                 df_high.head(5000).copy(deep=True)], sort=True)

nonrep = pd.concat([df_low.tail(5000).copy(deep=True),
                    df_high.tail(5000).copy(deep=True)], sort=True)

print('Rep: \n', rep['Above/Below 50K'].value_counts(), '\n')
print('Nonrep: \n', nonrep['Above/Below 50K'].value_counts())

nonrep['label'] = 1
rep['label'] = 0

del df, df_low, df_high

us = pd.concat([nonrep, rep], sort=True)

us_columns = list(us.columns)
meta = ['label', 'Above/Below 50K', 'index', 'bootstrap']
for m in meta:
    if m in us_columns:
        us_columns.remove(m)

us.reset_index(drop=True, inplace=True)
us.head()

In [ ]:
allensbach = pd.read_csv(os.path.join(path, 'data/allensbach_mrs.csv'))

allensbach.drop(['Unnamed: 0'], axis=1, inplace=True)

allensbach_columns = ['BRS1', 'BRS2', 'BRS3', 'BRS4', 'BRS5', 'BRS6', 
                      'Berufsgruppe', 'Erwerbstätigkeit', 'Geschlecht',
                      'Optimismus', 'Pessimismus', 'Schulabschluss', 'woechentlicheArbeitszeit']
allensbach.head()

In [22]:
gesis['Geschlecht'].values

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [29]:
gesis = pd.read_csv(os.path.join(path, 'data/gesis_processed.csv'), engine='python')
gbs = pd.read_csv(os.path.join(path, 'data/gbs_processed.csv'), engine='python')

gesis['label'] = 0
gbs['label'] = 1

cols = ['Geschlecht', 'Geburtsjahr', 'Geburtsland', 'Nettoeinkommen Selbst',
        'Nettoeinkommen Haushalt']

scaler = StandardScaler()
scaler.fit(gesis[cols].values)
gesis[cols] = scaler.transform(gesis[cols]) 
    
scaler = StandardScaler()
scaler.fit(gbs[cols].values)
gbs[cols] = scaler.transform(gbs[cols]) 
    
    
gesis_columns = ['Geschlecht', 'Aktiv', 'Berufsgruppe', 'Schlechter Schlaf', 'Leben genießen',
       'Zu Nichts aufraffen', 'Alles anstrengend', 'Wahlteilnahme',
       'Wach', 'Zurueckhaltend', 'Resilienz', 'Zufriedenheit Wahlergebnis',
       'leicht Vertrauen', 'Faulheit', 'Entspannt',
       'wenig kuenstlerisches Interesse', 'Gesellig',
       'Andere kritisieren', 'Geburtsland', 'Geburtsjahr',
       'Personen im Haushalt', 'Nettoeinkommen Selbst',
       'Nettoeinkommen Haushalt', 'Schlechter Schlaf', 'Leben genießen',
       'Zu Nichts aufraffen', 'Alles anstrengend', 'Wahlteilnahme',
       'Wahlabsicht', 'Desinteresse Politiker',
       'Nationalitaet', 'Familienstand', 'Hoechster Bildungsabschluss',
       'Berufliche Ausbildung', 'Erwerbstaetigkeit', 
       'Gruendlich', 'Nervoes', 'Phantasievoll', 'Optimismus Zukunft']

gesis.head()

c:\users\laksa\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if sys.path[0] == '':
c:\users\laksa\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  app.launch_new_instance()


,GESIS-CODE,Geschlecht,Geburtsjahr,Geburtsland,Nationalitaet,Familienstand,Hoechster Bildungsabschluss,Berufliche Ausbildung,Erwerbstaetigkeit,Berufsgruppe,...,Gesellig,Andere kritisieren,Gruendlich,Nervoes,Phantasievoll,Druck,Optimismus Zukunft,Zufriedenheit Wahlergebnis,Resilienz,label
0,288506501,-1.043054,-1.477462,0.339876,1.0,1.0,3.0,5.0,2.0,0.0,...,4.0,3.0,4.0,3.0,4.0,6.0,4.0,4.0,3.0,0
1,577577701,0.958724,0.474939,0.339876,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,3.0,4.0,5.0,1.0,8.0,3.0,3.0,4.0,0
2,358477201,0.958724,1.869510,0.339876,1.0,2.0,2.0,2.0,2.0,0.0,...,4.0,3.0,4.0,3.0,4.0,6.0,4.0,3.0,1.0,0
3,168186301,-1.043054,-1.198547,0.339876,1.0,1.0,3.0,2.0,1.0,0.0,...,3.0,3.0,4.0,3.0,4.0,8.0,4.0,3.0,5.0,0
4,212460601,-1.043054,1.590596,0.339876,1.0,2.0,2.0,1.0,1.0,0.0,...,2.0,4.0,4.0,5.0,3.0,6.0,4.0,3.0,2.0,0


In [ ]:
for C in [0.001, 0.01, 0.05, 0.1, 0.2, 0.5, 1, 1.5, 2]:
    for t in [0.01, 0.05, 0.1, 0.2, 0.5, 1]:

        auc = MRS(nonrep=gbs, rep=gesis, columns=cols,
                  temperature=t, ensemble_size=50, n_drop=100, limit=10, C=C)

        plt.plot([a[0] for a in auc], [a[1] for a in auc])
        plt.plot([a[0] for a in auc], len(auc)*[0.5], linestyle='--')
        plt.xlabel("removed instances")
        plt.title('SVM-C: ' + str(C) + ", Temperature: " + t)
        plt.grid()

plt.plot()

579, 573, 565, 561, 560, 554, 549, 547, 545, 544, 538, 537, 536, 534, 530, 523, 520, 516, 509, 505, 503, 498, 497, 496, 495, 493, 492, 490, 487, 484, 482, 474, 470, 466, 462, 459, 456, 455, 453, 447, 444, 441, 436, 433, 432, 430, 428, 427, 424, 423, 417, 414, 412, 409, 407, 405, 402, 395, 394, 393, 392, 389, 388, 384, 379, 376, 373, 369, 366, 361, 358, 356, 355, 347, 346, 343, 340, 339, 337, 335, 333, 331, 329, 326, 323, 320, 316, 315, 310, 305, 301, 299, 296, 292, 289, 283, 281, 275, 273, 270, 268, 265, 262, 261, 260, 255, 254, 253, 251, 246, 245, 240, 239, 237, 235, 233, 231, 230, 229, 226, 225, 223, 219, 